In [4]:
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image

# ======================
# SETTINGS
# ======================
MODEL_PATH = "best_model.pth"   # path to your .pth model
IMAGE_PATH = "Screenshot 2025-10-03 152318.png"         # path to the image you want to classify
DEVICE = "cpu"                  # change to "cuda" if GPU available
CLASS_NAMES = ["Drug User", "Not Drug User"]

# ======================
# LOAD MODEL
# ======================
def load_model(model_path, device):
    # Load the saved weights
    checkpoint = torch.load(model_path, map_location=device)

    # Build EfficientNet-B0 for 2-class classification
    model = models.efficientnet_b0(pretrained=False)
    model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 2)

    # Remove "module." prefix if trained with DataParallel
    new_state_dict = {}
    for k, v in checkpoint.items():
        new_k = k.replace("module.", "") if k.startswith("module.") else k
        new_state_dict[new_k] = v

    model.load_state_dict(new_state_dict)
    model.to(device)
    model.eval()
    return model

# ======================
# PREPROCESS IMAGE
# ======================
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
    img = Image.open(image_path).convert("RGB")
    return transform(img).unsqueeze(0)

# ======================
# PREDICT
# ======================
def predict(model, image_tensor, device):
    with torch.no_grad():
        outputs = model(image_tensor.to(device))
        probs = F.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probs, 1)
        return CLASS_NAMES[predicted.item()], confidence.item()

# ======================
# MAIN
# ======================
if __name__ == "__main__":
    model = load_model(MODEL_PATH, DEVICE)
    image_tensor = preprocess_image(IMAGE_PATH)
    label, confidence = predict(model, image_tensor, DEVICE)

    print(f"\n🧠 Prediction Result:")
    print(f"   ➤ Label: {label}")
    print(f"   ➤ Confidence: {confidence * 100:.2f}%\n")




🧠 Prediction Result:
   ➤ Label: Drug User
   ➤ Confidence: 99.01%

